In [5]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
import autopep8
import pandas as pd

In [2]:
FILENAME='phi-2.Q5_0.gguf'
model_path=f'./model/{FILENAME}'
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
python_repl = PythonREPL()

In [3]:
llm = LlamaCpp(
    model_path=model_path,
    repetition_penalty= 2.0,
    temperature=0.00,
    do_sample=True,
    max_tokens=2000,
    n_ctx=2048,
    context_length=4000,
    callback_manager=callback_manager,
    top_k=50,
    top_p=0.95,
)

D:\01MyFolder\mypython\python-3.11.5-embed-amd64\repository\jpynb\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! repetition_penalty is not default parameter.
                repetition_penalty was transferred to model_kwargs.
                Please confirm that repetition_penalty is what you intended.
  warnings.warn(
D:\01MyFolder\mypython\python-3.11.5-embed-amd64\repository\jpynb\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! do_sample is not default parameter.
                do_sample was transferred to model_kwargs.
                Please confirm that do_sample is what you intended.
  warnings.warn(
D:\01MyFolder\mypython\python-3.11.5-embed-amd64\repository\jpynb\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! context_length is not default parameter.
                context_length was transferred to model_kwargs.
                Please confirm that context_length is what you intended.
  warnings.w

In [14]:
df=pd.read_csv('./data/sales.csv')
df

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,5/28/2010,669165933,6/27/2010,9925,255.28,159.42,2533654.00,1582243.50,951410.50
1,Central America and the Caribbean,Grenada,Cereal,Online,C,8/22/2012,963881480,9/15/2012,2804,205.70,117.11,576782.80,328376.44,248406.36
2,Europe,Russia,Office Supplies,Offline,L,5/2/2014,341417157,5/8/2014,1779,651.21,524.96,1158502.59,933903.84,224598.75
3,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Online,C,6/20/2014,514321792,7/5/2014,8102,9.33,6.92,75591.66,56065.84,19525.82
4,Sub-Saharan Africa,Rwanda,Office Supplies,Offline,L,2/1/2013,115456712,2/6/2013,5062,651.21,524.96,3296425.02,2657347.52,639077.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sub-Saharan Africa,Mali,Clothes,Online,M,7/26/2011,512878119,9/3/2011,888,109.28,35.84,97040.64,31825.92,65214.72
96,Asia,Malaysia,Fruits,Offline,L,11/11/2011,810711038,12/28/2011,6267,9.33,6.92,58471.11,43367.64,15103.47
97,Sub-Saharan Africa,Sierra Leone,Vegetables,Offline,C,6/1/2016,728815257,6/29/2016,1485,154.06,90.93,228779.10,135031.05,93748.05
98,North America,Mexico,Personal Care,Offline,M,7/30/2015,559427106,8/8/2015,5767,81.73,56.67,471336.91,326815.89,144521.02


# FewShotPromptTemplate with examples

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
examples = [
      {
        "question": "get the distinct region",
        "answer":
        """
        ```python
        output = df['region'].unique()
        ```
        
        """
      },
    {
        "question": "get the record with max unit_sold",
        "answer":
        """
        ```python
        output = df[df['Units Sold']==df['Units Sold'].max()]
        ```
        
        """
      },


    ]
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")
prompt_with_example = FewShotPromptTemplate(
                            examples=examples,
                            example_prompt=example_prompt,
                            suffix="Question: {input}",
                            input_variables=["input"]
                        )
agent = LLMChain(llm=llm,prompt=prompt_with_example)

# pass the prompt to the LLM

In [8]:
query = "Change the value in sales channel as Offline to 0 and Online to 1"
columns=df.columns.tolist()
instruction = """
As a python coder create a pythonic response for the query with reference to the columns in my pandas dataframe{columns}.
Instruction:
Do not write the whole script just give me a pythonic response for this query and do not extend more than asked.
Include a print statement for the final output.
question: {input}

answer:

"""
prompt_template = PromptTemplate.from_template(instruction)
prompt = prompt_template.format(input=query,columns=columns)
response = agent.invoke(prompt)

  ```python
  df['Sales Channel'] = df['Sales Channel'].replace({'Offline':0, 'Online':1})
  print(df)
  ```


# Format the response

In [9]:
def extract_code(response):
    start = 0
    q = ""
    temp_block=""
    for line in response.splitlines(): #response['text'].splitlines():
        if '```python' in line and start==0:
            start=1
        if '```' == line and start==1:
            break
        if start ==1 and '```' not in line:
            q=q+'\n'+line
            #print(line)
    #print(q)
    return q

In [10]:
q=extract_code(response['text'])
gencode=autopep8.fix_code(q)

In [12]:
code=f"""
import pandas as pd

df=pd.read_csv('./data/sales.csv')

{gencode}


"""
print(autopep8.fix_code(code))


import pandas as pd

df = pd.read_csv('./data/sales.csv')


df['Sales Channel'] = df['Sales Channel'].replace({'Offline': 0, 'Online': 1})
print(df)



# Excute the generated code

In [13]:
output=python_repl.run(code)
print(output)

Python REPL can execute arbitrary code. Use with caution.


                               Region                Country        Item Type  \
0               Australia and Oceania                 Tuvalu        Baby Food   
1   Central America and the Caribbean                Grenada           Cereal   
2                              Europe                 Russia  Office Supplies   
3                  Sub-Saharan Africa  Sao Tome and Principe           Fruits   
4                  Sub-Saharan Africa                 Rwanda  Office Supplies   
..                                ...                    ...              ...   
95                 Sub-Saharan Africa                   Mali          Clothes   
96                               Asia               Malaysia           Fruits   
97                 Sub-Saharan Africa           Sierra Leone       Vegetables   
98                      North America                 Mexico    Personal Care   
99                 Sub-Saharan Africa             Mozambique        Household   

    Sales Channel Order Pri

<string>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
